# Setup

In [ ]:
from anastruct import SystemElements
import difflib
import numpy as np
import matplotlib.pyplot as plt
import copy

ss = SystemElements()
colors = {
    "node_number": "red",
    "element_number": "green",
}

ss.change_plot_colors(plot_colors=colors)

# Gerador de equações

traducao = {
    "axial_force": "força normal",
    "bending_moment": "momento fletor",
    "shear_force": "força cortante",
    "axial_tension": "tensão normal",
}


def plot_equation(x, y):
    for i in range(len(y)):
        y[i] = round(y[i], 4)
    plt.plot(x, y, "o", label="Dados originais")
    plt.plot(x, y, "-", label=f"Ajuste polinomial")
    plt.xlabel("Comprimento do Elemento")
    plt.ylabel("Força Axial")
    plt.title("Distribuição da Força Axial ao Longo do Elemento")
    plt.legend()
    plt.grid(True)
    plt.show()


def get_equations(diagram="bending_moment", show_plots=False):
    for j in range(1, len(ss.element_map) + 1):
        el = ss.element_map[j]

        if diagram == "axial_force":
            y = copy.deepcopy(el.axial_force)
        elif diagram == "bending_moment":
            y = copy.deepcopy(el.bending_moment)
        elif diagram == "shear_force":
            y = copy.deepcopy(el.shear_force)
        else:
            print("Entrada Inválida!")
            return

        x = np.linspace(0.0, el.l, len(y))

        degree = 10
        coefficients = np.polyfit(x, y, degree)
        polynomial = np.poly1d(coefficients)

        # Imprimir a equação do polinômio
        def polynomial_equation(coefficients):
            terms = ["y ="]
            degree = len(coefficients) - 1

            for i, coeff in enumerate(coefficients):
                if abs(coeff) > 1e-5 or i == 10:
                    # Determine the sign and the absolute value of the coefficient
                    sign = "+" if coeff > 0 else "-"
                    abs_coeff = abs(coeff)
                    if abs_coeff < 1e-5:
                        sign = "+"

                    if len(terms) == 1:
                        if degree - i == 0:
                            if sign == "-" and abs_coeff > 1e-5:
                                term = f"{sign} {abs_coeff:.2f}".rstrip("0").rstrip(".")
                            else:
                                term = f"{abs_coeff:.2f}".rstrip("0").rstrip(".")
                        elif degree - i == 1:
                            if sign == "-" and abs_coeff > 1e-5:
                                term = (
                                    f"{sign} {abs_coeff:.2f}".rstrip("0").rstrip(".")
                                    + "x"
                                )
                            else:
                                term = f"{abs_coeff:.2f}".rstrip("0").rstrip(".") + "x"
                        else:
                            if sign == "-" and abs_coeff > 1e-5:
                                term = (
                                    f"{sign} {abs_coeff:.2f}".rstrip("0").rstrip(".")
                                    + f"x^{degree - i}"
                                )
                            else:
                                term = (
                                    f"{abs_coeff:.2f}".rstrip("0").rstrip(".")
                                    + f"x^{degree - i}"
                                )
                    else:
                        if degree - i == 0:
                            if abs_coeff < 1e-5:
                                continue
                            term = f"{sign} {abs_coeff:.2f}".rstrip("0").rstrip(".")
                        elif degree - i == 1:
                            term = (
                                f"{sign} {abs_coeff:.2f}".rstrip("0").rstrip(".") + "x"
                            )
                        else:
                            term = (
                                f"{sign} {abs_coeff:.2f}".rstrip("0").rstrip(".")
                                + f"x^{degree - i}"
                            )
                    if term.startswith("1x"):
                        term = "x" + term[2:]
                    terms.append(term)

            # Join the terms and adjust the first sign if needed
            polynomial = " ".join(terms)
            if polynomial.startswith("+ "):
                polynomial = polynomial[2:]

            return polynomial

        equation = polynomial_equation(coefficients)
        print(f"Equação do polinômio ajustado de {traducao[diagram]} da barra {j}:")
        print(equation)
        if show_plots:
            plot_equation(x, y)

# Tensáo normal
def show_axial_tension():
    id= int(input(f"Insira a barra em que você quer calcular a tensão normal: "))
    el = ss.element_map[id]
    x = float(
        input(
            f"Insira distância a partir do nó inicial da barra do ponto onde você quer calcular a tensão normal (de 0 a {el.l:.2f}): "
        ).replace(",", ".")
    )
    index_axial_force = round(x/el.l*len(el.axial_force))
    index_bending_moment = round(x/el.l*len(el.bending_moment))

    axial_force = el.axial_force[index_axial_force]
    bending_moment = el.bending_moment[index_bending_moment]

    a = float(input(f"Insira a área da seção da barra {id}: ").replace(",", "."))
    I_y = float(
        input(
            f"Insira o momento de inércia em relação ao eixo y (como convencionado em sala) da seção da barra {id}: "
        ).replace(",", ".")
    )
    h = float(input(f"Insira a altura da seção da barra {id}: ").replace(",", "."))
    g = float(
        input(
            f"Insira a posição do baricentro em relação ao ponto mais baixo da seção da barra {id}: "
        ).replace(",", ".")
    )

    def f(z):
        x1 = axial_force / a
        x2 = bending_moment * z / I_y
        return x1 + x2

    y_points = np.array([-g, 0, h-g])
    x_points = np.array(f(y_points))

    # Plotando o diagrama
    plt.plot(x_points, y_points, "o-")

    # Fill the area for x >= 0
    plt.fill_betweenx(y_points, x_points, where=(x_points >= 0), color='red', alpha=0.5, interpolate=True, label='Tração')

    # Fill the area for x < 0
    plt.fill_betweenx(y_points, x_points, where=(x_points < 0), color='blue', alpha=0.5, interpolate=True, label='Compressão')

    # Adicionando valores de x sobre cada ponto
    for i in range(len(x_points)):
        plt.text(
            x_points[i],
            y_points[i],
            f"G (E={x_points[i]:.2f}, h={y_points[i]:.2f})" if (y_points[i] < 1e-5) else f"(E={x_points[i]:.2f}, h={-y_points[i]:.2f})",
            fontsize=9,
            ha="right",
        )

    # Configurando o gráfico
    plt.axvline(0, color="black", linewidth=1)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.title("Diagrama de tensão normal")

    # Esconder grid e eixos
    plt.axis('off')

    plt.show()

# Inputs

##### Exemplo Feito

In [ ]:
# Exemplo Feito 1
# Barras location = [x1, y1], [x2, y2]
ss.add_element(location=[[0, 0], [0, 2]])  # Barra 1
ss.add_element(location=[[0, 2], [0, 4]])  # Barra 2
ss.add_element(location=[[0, 4], [3, 4]])  # Barra 3
ss.add_element(location=[[3, 4], [4, 4]])  # Barra 4
ss.add_element(location=[[4, 4], [6, 4]])  # Barra 5
ss.add_element(location=[[6, 4], [7, 4]])  # Barra 6

ss.add_internal_hinge(node_id=3)
ss.add_internal_hinge(node_id=5)

ss.add_support_fixed(node_id=1)
ss.add_support_roll(node_id=4)
ss.add_support_roll(node_id=6)

ss.point_load(node_id=2, Fx=+5)
ss.q_load(q=-2, element_id=3)
ss.q_load(q=-2, element_id=4)
ss.point_load(node_id=7, Fy=-10)

In [ ]:
# Exemplo Feito 2
ss.add_element(location=[[0, 0], [3, 4]])  # Barra 1
ss.add_element(location=[[3, 4], [5, 4]])  # Barra 2
ss.add_element(location=[[5, 4], [7, 4]])  # Barra 3
ss.add_element(location=[[7, 4], [9, 4]])  # Barra 4
ss.add_element(location=[[7, 4], [7, 2]])  # Barra 5
ss.add_element(location=[[7, 2], [7, -2]])  # Barra 6

ss.add_support_hinged(node_id=1)  # Apoio simples 1
ss.add_support_hinged(node_id=7)  # Apoio simples 2
ss.add_internal_hinge(node_id=3)  # Articulação

ss.point_load(node_id=5, Fy=-10)  # Carga no nó
ss.q_load(q=-5, element_id=2)  # Carga distribuída
ss.q_load(q=-5, element_id=3)  # Carga distribuída
ss.moment_load(node_id=6, Tz=2)  # Momento no nó
ss.show_structure()

##### Input do Usuário

In [ ]:
# Adicione uma barra
apoio = input("Digite o ponto de início da barra x1 y1: ")
apoio += " " + input("Digite o ponto de fim da barra x2 y2: ")
apoio = apoio.split(" ")
x1, y1, x2, y2 = float(apoio[0]), float(apoio[1]), float(apoio[2]), float(apoio[3])
ss.add_element(location=[[x1, y1], [x2, y2]])

ss.show_structure()

In [ ]:
# Adicione um apoio
apoio = input(
    "Digite um tipo de apoio entre (apoio simples, apoio móvel, articulação, engaste): "
)

# Lista de opções válidas
apoios_validos = ["apoio simples", "apoio móvel", "articulação", "engaste"]


# Função para encontrar a opção mais próxima
def find_closest_option(user_input, options):
    closest_matches = difflib.get_close_matches(user_input, options, n=1, cutoff=0.6)
    if closest_matches:
        return closest_matches[0]
    else:
        return None


# Encontrar a opção mais próxima
apoio = find_closest_option(apoio, apoios_validos)

if apoio == None:
    print("Apoio inválido. Tente novamente.")

elif apoio == "apoio simples":
    id = input("Adicione o ID do nó onde deseja colocar o suporte: ")
    id = int(id)
    ss.add_support_hinged(node_id=id)

elif apoio == "apoio móvel":
    id = int(input("Adicione o ID do nó onde deseja colocar o suporte: "))
    direction = input("Adicione a direção sem restrição de movimento (x ou y): ")
    ss.add_support_roll(node_id=id, direction=2 if direction == "x" else 1)

elif apoio == "articulação":
    id = input("Adicione o ID do nó onde deseja colocar o suporte: ")
    id = int(id)
    ss.add_internal_hinge(id)

elif apoio == "engaste":
    id = int(input("Adicione o ID do nó onde deseja colocar o suporte: "))
    ss.add_support_fixed(id)

ss.show_structure()

In [ ]:
# Adicione uma carga distribuída a uma barra
id = int(input("Insira o ID da barra: "))
q = float(input("Insira a carga distribuída q: ").replace(",", "."))

ss.q_load(q=q, element_id=id)
ss.show_structure()

In [ ]:
# Adicione uma força a um nó
id = int(input("Insira o ID do nó: "))
Fx = float(input("Insira o Fx: ").replace(",", "."))
Fy = float(input("Insira o Fy: ").replace(",", "."))

ss.point_load(node_id=id, Fx=Fx, Fy=Fy)
ss.show_structure()

In [ ]:
# Adicione um momento a um nó
id = int(input("Insira o ID do nó: "))
Tz = float(input("Insira o M: ").replace(",", "."))

ss.moment_load(node_id=id, Tz=Tz)
ss.show_structure()

# Resultados

In [ ]:
# Mostrar estrutura
ss.show_structure()

In [ ]:
# Calcular
ss.solve()

In [ ]:
# Plot resultados gerais
ss.show_results()

In [ ]:
# Reações de Apoio
for node in ss.reaction_forces:
    print(f"No nó {node}:")
    print(f"\tFx = {-ss.get_node_results_system(node_id=node)['Fx']:.2f}")
    print(f"\tFy = {-ss.get_node_results_system(node_id=node)['Fy']:.2f}")


ss.show_reaction_force()

In [ ]:
# Normal
get_equations(diagram="axial_force", show_plots=False)
ss.show_axial_force()

In [ ]:
# Tensão Normal
show_axial_tension()

In [ ]:
# Cortante
get_equations(diagram="shear_force", show_plots=False)
ss.show_shear_force()

In [ ]:
# Momento Fletor
get_equations(diagram="bending_moment", show_plots=False)
ss.show_bending_moment()